# core

> Fill in a module description here

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()

In [ ]:
#| export
from dataclasses import dataclass

from exex.imports import *
from exex.utils import *

In [ ]:
#| export
class Event(list):
    def __call__(self, *args, **kwargs):
        for item in self:
            item(*args, **kwargs)

### Unit

In [ ]:
#| export
@dataclass
class Unit:
    """
    Default Units
    """
    
    # SI Unit
    LENGTH = 'meter'
    MASS = 'kilogram'
    TIME = 'second'
    TEMPERATURE = 'kelvin'
    
    # Derived from SI Unit
    MOLE = 'mole'
    SPECIFIC_HEAT = 'joule / (kilogram kelvin)'
    PRESSURE = 'pascal'
    VOLUME = 'liter'

In [ ]:
#| hide
test_eq(Unit.MASS, 'kilogram')

In [ ]:
#| export
ureg = pint.UnitRegistry(system='SI')
Q = ureg.Quantity # quantity

In [ ]:
@patch
def to_standard(self: Q): return 1

In [ ]:
moles = Q(0.214, Unit.MOLE)

In [ ]:
moles

0.214 <Unit('mole')>

In [ ]:
moles.__dict__

{'_magnitude': 0.214,
 '_units': <UnitsContainer({'mole': 1})>,
 '_Quantity__used': False,
 '_Quantity__handling': None}

In [ ]:
moles.dimensionality

<UnitsContainer({'[substance]': 1})>

In [ ]:
#| hide
test_eq(moles.magnitude, 0.214) 
test_eq(moles.units, "mole")

In [ ]:
specific_heat = Q(0.235, Unit.SPECIFIC_HEAT)

In [ ]:
specific_heat.ito_reduced_units()

In [ ]:
#| hide
test_eq(specific_heat.magnitude, 0.235)
test_eq(specific_heat.units, 'joule/(kelvin kilogram)')

In [ ]:
m = Q('3.21 kilogram')

In [ ]:
m

3.21 <Unit('kilogram')>

In [ ]:
@ureg.check('[length]')
def foo(length):
    return length

In [ ]:
foo('10 meter')

'10 meter'

In [ ]:
@ureg.wraps((ureg.meter, ureg.gram), (ureg.meter, ureg.kilogram))
def mypp(length, mass):
    return length, mass

In [ ]:
mypp('10 kilometer', '1 kilogram')

(10000.0 <Unit('meter')>, 1 <Unit('gram')>)

In [ ]:
@ureg.check('[mass]')
@ureg.wraps(None, None)
def set_mass(mass):
    return mass

In [ ]:
set_mass('20 kilogram')

'20 kilogram'

In [ ]:
@ureg.check('[mass]')
@ureg.wraps(ureg.gram, ureg.kilogram)
def set_mass(mass):
    return mass

In [ ]:
set_mass('20 kilogram')

20 <Unit('gram')>

### Data

In [ ]:
#| export
class Object:
    pass

In [ ]:
#| export
class PropertyData(dict):
    pass

In [ ]:
c = PropertyData({'0': {'mass': 2, 'object': Object()}})

In [ ]:
c['0']

{'mass': 2, 'object': <__main__.Object>}

In [ ]:
c['1'] = {'mass': 2.1, 'object': 'XXX'}

### Proprety

In [ ]:
#| export
class PropertyObservable:
    def __init__(self, compound):
        self._data = PropertyData()
        self.compound = compound
        self._connections = []
        self.laws = dict()
        self.func_changed = Event()
        
        # print(f'self.compound={self.compound}')
        # print(self.compound.__dict__)
        # print(f'formula={self.compound.formula}')
        
        self.symbol = smp.symbols(f'{self.abbreviate}', real=True)
    
    @property
    def name(self) -> str:
        return camel_to_snake(self.__class__.__name__)
    
    @classmethod
    @property
    def snake_name(cls) -> str: # return the snake style name
        return camel_to_snake(cls.__name__)
    
    def add_data(self, time, value):
        self._data[time] = {'value': value}
    
    def get_value(
        self,
        time: int # time
    ):
        if not time in self._data:
            return f"Don't have data for time={time}"
        return self._data[time]['value']
    
    def add_law(self, law):
        if not law in self.laws:
            self.laws[camel_to_snake(law.__class__.__name__)] = law

In [ ]:
#| export
class Mass(PropertyObservable):
    def __init__(self, compound):
        self.abbreviate = 'm'
        self.unit = Unit.MASS
        super().__init__(compound)

In [ ]:
#| export
@patch
def set_mass(self: Mass):
    pass

In [ ]:
#| export
class Mole(PropertyObservable):
    def __init__(self, compound):
        self.abbreviate = 'n'
        self.unit = Unit.MOLE
        super().__init__(compound)

In [ ]:
#| export
class Pressure(PropertyObservable):
    def __init__(self, compound):
        self.abbreviate = 'P'
        self.unit = Unit.PRESSURE
        super().__init__(compound)

In [ ]:
#| export
class Volume(PropertyObservable):
    def __init__(self, compound):
        self.abbreviate = 'V'
        self.unit = Unit.VOLUME
        super().__init__(compound)

In [ ]:
#| export
class Temperature(PropertyObservable):
    def __init__(self, compound):
        self.abbreviate = 'T'
        self.unit = Unit.TEMPERATURE
        super().__init__(compound)

### Law

In [ ]:
#| export
class Law:
    
    @property
    def name(self) -> str:
        return camel_to_snake(self.__class__.__name__)
    
    @classmethod
    @property
    def snake_name(cls) -> str: # return the snake style name
        return camel_to_snake(cls.__name__)
    
    def n_known_variables(
        self,
        timestep: int
    ) -> int: # the number of known variables
        n_knowns = 0
        for p in self.properties:
            name = camel_to_snake(p['object'].__name__)
            if timestep in self.compound.properties[name]._data:
                n_knowns += 1
        return n_knowns

    def is_solveable(
        self,
        timestep: int # the timestep
    ) -> bool:
        n_unknowns = len(self.properties)
        n_knowns = self.n_known_variables(timestep)
        print(f'n_unknowns={n_unknowns} and n_knowns={n_knowns}')
        return n_unknowns - n_knowns <= 1

    def _run_config(self) -> None: # run all configuration
        self._config_properties()
    
    def _config_properties(self) -> None: # add law's properties to compound
        
        for p in self.properties:
            # name = camel_to_snake(p['object'].__name__)
            name = p['object'].snake_name
            
            if not name in self.compound.properties:
                self.compound.properties[name] = p['object'](compound=self.compound)
            
            self.compound.properties[name].add_law(self)
    
    def solve(
        self,
        timestep: int, # the time step of the unknown variable
        unknown: str # the unknown variable that you want to solve
    ):
        unknown_symbol = self.compound.properties[unknown].symbol
        return smp.solve(self.expression, unknown_symbol)

In [ ]:
class Law:
    def __init__(self):
        self._properties: dict = []
        self._laws: dict = []
    
    def relation(self):
        #self._laws['x']
        pass

### System

System is an mediator that compounds and reactions communicate through each others.

In [ ]:
#| export
class System:
    def __init__(self):
        self.reactions = dict()
        self.universe = None
        self.subscribers: dict[str, Callable] = dict()
        self.current_time: int = None
        self.highest_time: int = None